In [5]:
import fitz  # PyMuPDF
import os
from pathlib import Path
import re
import pytesseract
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [6]:
def limpar_nome(nome):
    return re.sub(r'[\\/*?:"<>|]', "_", nome)

# Função para buscar o valor após uma palavra-chave
def encontrar_valor(texto, chave, linha_abaixo=False):
    linhas = texto.split("\n")
    for i, linha in enumerate(linhas):
        if chave in linha:
            if linha_abaixo and i + 1 < len(linhas):
                return linhas[i + 1].strip()
            else:
                return linha.split(chave)[-1].strip()
    return "Não encontrado"

# Função para ajustar os nomes dos arquivos PDF
def ajustar_nomes():
    diretorio = 'M:\\Laudos Anteriores Itau\\'
    lista_arquivos = os.listdir(diretorio)

    for arquivo in lista_arquivos:
        if not arquivo.lower().endswith(".pdf"):
            continue  # Ignorar arquivos que não são PDF

        caminho_completo = os.path.join(diretorio, arquivo)
        print(f"Processando: {caminho_completo}")

        try:
            with fitz.open(caminho_completo) as doc:
                texto_pg1 = doc[0].get_text()
                uf = encontrar_valor(texto_pg1, "UF")
                cidade = encontrar_valor(texto_pg1, "Cidade")
                matricula = encontrar_valor(texto_pg1, "Matrícula")
                bairro = encontrar_valor(texto_pg1, "Bairro/Setor")
                num_controle = encontrar_valor(texto_pg1, "Nº Controle Interno / Ordem de Serviço")

                texto_pg2 = doc[1].get_text()
                data_laudo = encontrar_valor(texto_pg2, "Data Elaboração Laudo", linha_abaixo=True)
                if data_laudo == "Não encontrado":
                    texto_pg3 = doc[2].get_text()
                    data_laudo = encontrar_valor(texto_pg3, "Data Elaboração Laudo", linha_abaixo=True)

            uf = limpar_nome(uf)
            cidade = limpar_nome(cidade)
            bairro = limpar_nome(bairro)
            num_controle = limpar_nome(num_controle)
            data_laudo = limpar_nome(data_laudo)

            # Renomear o arquivo
            path = Path(caminho_completo)
            novo_nome = f'{uf} - {cidade} - {bairro} - {matricula} - {num_controle} - {data_laudo}.pdf'
            novo_caminho = path.with_name(novo_nome)

            path.rename(novo_caminho)
            print(f"✅ Arquivo renomeado para: {novo_caminho}")

        except PermissionError as e:
            print(f"🚫 Erro de permissão ao renomear '{arquivo}': {e}")
        except Exception as e:
            print(f"⚠️ Erro ao processar '{arquivo}': {e}")
            
# Função para encontrar o número da residência
def encontrar_numero_residencia(texto):
    # Divide o texto em linhas
    linhas = texto.split('\n')
    for linha in linhas:
        # Verifica se tem "N°" ou "Nº" sozinho seguido de número
        if re.search(r'\bN[º°]\b', linha):
            if not any(palavra in linha.lower() for palavra in ['pavimento', 'cartório', 'elevador', 'unidade', 'condomínio', 'ofício']):
                match = re.search(r'\bN[º°] ?[:\-]? ?(\d{1,5})\b', linha)
                if match:
                    return match.group(1)
    return "Não encontrado"

# Função para encontrar o valor de outras paginas
def outras_linhas(valor, doc, texto):
    if valor == "Não encontrado":
        texto_pg3 = doc[1].get_text()
        valor = encontrar_valor(texto_pg3, f"{texto}", linha_abaixo=True)
    elif valor == "Não encontrado":
        texto_pg3 = doc[2].get_text()
        valor = encontrar_valor(texto_pg3, f"{texto}", linha_abaixo=True)
    
    return valor


In [7]:
def scrapping_pdf():
    """
    Função para pegar dados dentro do PDF padrão Itaú.
    """
    # Defina o caminho do diretório onde os arquivos PDF estão localizados
    diretorio = 'M:\\Thiago\\WebScrapping\\teste_pdf\\'
    # Obtenha a lista de arquivos no diretório
    lista_arquivos = os.listdir(diretorio)

    # Itera sobre cada arquivo no diretório
    for arquivo in lista_arquivos:
        # Verifica se o arquivo é um PDF (ignorando maiúsculas/minúsculas)
        if not arquivo.lower().endswith(".pdf"):
            continue  # Ignora arquivos que não são PDF

        # Cria o caminho completo do arquivo
        caminho_completo = os.path.join(diretorio, arquivo)


        try:
            # Abre o arquivo PDF usando PyMuPDF
            with fitz.open(caminho_completo) as doc:

                # Informações do Imóvel Avaliando
                texto_pg1 = doc[0].get_text()
                num_controle = encontrar_valor(texto_pg1, "Nº Controle Interno / Ordem de Serviço")
                valor_compra_e_venda = encontrar_valor(texto_pg1, "Valor Compra Venda")
                matricula = encontrar_valor(texto_pg1, "Matrícula")
                logradouro = encontrar_valor(texto_pg1, "Logradouro")
                numero = encontrar_numero_residencia(texto_pg1)
                andar = encontrar_valor(texto_pg1, "Andar")
                complemento = encontrar_valor(texto_pg1, "Complemento")
                bairro = encontrar_valor(texto_pg1, "Bairro/Setor")
                cidade = encontrar_valor(texto_pg1, "Cidade")
                uf = encontrar_valor(texto_pg1, "UF")
                latitude = encontrar_valor(texto_pg1, "Latitude").split(' ')[0].strip()
                longitude = encontrar_valor(texto_pg1, "Longitude")
                coordenadas = encontrar_valor(texto_pg1, "Coordenadas do imóvel:")
                idade = encontrar_valor(texto_pg1, "Anos")
                if coordenadas == "Não encontrado":
                    texto_pg3 = doc[1].get_text()
                    coordenadas = encontrar_valor(texto_pg3, "Coordenadas do imóvel:")
                
                # Dimensões do Imóvel Avaliando
                areatotal = encontrar_valor(texto_pg1, "Área Total:")
                testada = encontrar_valor(texto_pg1, "Testada (Frente):")
                fracao = encontrar_valor(texto_pg1, "Fração Ideal:")

                # Edifício ao qual pertence o Imóvel Avaliando
                pavimentos = encontrar_valor(texto_pg1, "N° de Pavimentos:")
                unidades_por_andar = encontrar_valor(texto_pg1, "N° Unidades Por Andar:")
                total_unindades_condominio = encontrar_valor(texto_pg1, "condomínio:", linha_abaixo=True)
                qtd_elevadores = encontrar_valor(texto_pg1, "N° de Elevadores:")
                descricao_andares_pavimentos = encontrar_valor(texto_pg1, "Andares/Pavimentos")
                uso_edificio = encontrar_valor(texto_pg1, "Uso do Edifício:")
                uso_imovel_avaliando = encontrar_valor(texto_pg1, "Uso do Imóvel Avaliando:")
                vagas_cobertas = encontrar_valor(texto_pg1, "Cobertas").split(' ')[0].strip()
                vagas_descobertas = encontrar_valor(texto_pg1, "Descobertas").split(' ')[2].strip()
                vagas_privativas = encontrar_valor(texto_pg1, "Privativas").strip()
                fechamento_paredes = encontrar_valor(texto_pg1, "Fechamento das Paredes:")
                total_banheiros = encontrar_valor(texto_pg1, "Total de Banheiros:")
                fachada_principal = encontrar_valor(texto_pg1, "Fachada Principal")
                esquadrias = encontrar_valor(texto_pg1, "Esquadrias")
                num_pavimentos_unidade = encontrar_valor(texto_pg1, "N° Pavimentos da Unidade")
                num_dormitorios = encontrar_valor(texto_pg1, "Nº Dormitórios")
                area_privativa = encontrar_valor(texto_pg1, "Área Privativa:", linha_abaixo=True)
                area_comum = encontrar_valor(texto_pg1, "Área Comum (m²):", linha_abaixo=True)
                area_total = encontrar_valor(texto_pg1, "Área Total (m²):", linha_abaixo=True)
                area_averbada = encontrar_valor(texto_pg1, "Área Averbada:")
                area_nao_averbada = encontrar_valor(texto_pg1, "Área não Averbada:")

                # Valores
                valor_mercado_total = encontrar_valor(texto_pg1, "Valor de Mercado Total do Imóvel:", linha_abaixo=True)
                outras_linhas(valor_mercado_total, doc, "Valor de Mercado Total do Imóvel:")
                valor_liquidez = encontrar_valor(texto_pg1, "Valor de Liquidez:")
                outras_linhas(valor_liquidez, doc, "Valor de Liquidez:")
                data_elaboração = encontrar_valor(texto_pg1, "Data Elaboração Laudo", linha_abaixo=True)
                outras_linhas(data_elaboração, doc, "Data Elaboração Laudo")

                print("-"*80)
                print(f"Processando: {caminho_completo}")
                print("Proposta: ", num_controle)
                print("Valor Compra e Venda: ", valor_compra_e_venda)
                print("Matrícula: ", matricula)
                print("Logradouro: ", logradouro)
                print("Número: ", numero)
                print("Andar: ", andar)
                print("Complemento: ", complemento)
                print("Bairro: ", bairro)
                print("Cidade: ", cidade)
                print("UF: ", uf)
                print("Latitude: ", latitude)
                print("Longitude: ", longitude)
                print("Coordenadas: ", coordenadas)
                print("Área Total: ", areatotal)
                print("Testada (Frente): ", testada)
                print("Fração Ideal: ", fracao)
                print("N° de Pavimentos: ", pavimentos)
                print("N° Unidades Por Andar: ", unidades_por_andar)
                print("Nº Total de Unidades no condomínio: ", total_unindades_condominio)
                print("N° de Elevadores: ", qtd_elevadores)
                print("Andares/Pavimentos: ", descricao_andares_pavimentos)
                print("Uso do Edifício: ", uso_edificio)
                print("Uso do Imóvel Avaliando: ", uso_imovel_avaliando)
                print("Cobertas: ", vagas_cobertas)
                print("Descobertas: ", vagas_descobertas)
                print("Privativas: ", vagas_privativas)
                print("Fechamento das Paredes: ", fechamento_paredes)
                print("Total de Banheiros: ", total_banheiros)
                print("Fachada Principal: ", fachada_principal)
                print("Esquadrias: ", esquadrias)
                print("N° Pavimentos da Unidade: ", num_pavimentos_unidade)
                print("Nº Dormitórios: ", num_dormitorios)
                print("Área Privativa: ", area_privativa)
                print("Área Comum: ", area_comum)
                print("Área Total: ", area_total)
                print("Área Averbada: ", area_averbada)
                print("Área não Averbada: ", area_nao_averbada)
                print("Valor de Mercado Total do Imóvel: ", valor_mercado_total)
                print("Valor de Liquidez: ", valor_liquidez)
                print("Data Elaboração Laudo: ", data_elaboração)
                print("-"*80)

        except PermissionError as e:
            print(f"🚫 Erro de permissão '{arquivo}': {e}")
        except Exception as e:
            print(f"⚠️ Erro ao processar '{arquivo}': {e}")

def ler_paginas():
    diretorio = 'M:\\Thiago\\WebScrapping\\teste_pdf\\'
    lista_arquivos = os.listdir(diretorio)

    for arquivo in lista_arquivos:
        if not arquivo.lower().endswith(".pdf"):
            continue

        caminho_completo = os.path.join(diretorio, arquivo)
        print(f"Processando: {caminho_completo}")

        try:
            with fitz.open(caminho_completo) as doc:
                for i, pagina in enumerate(doc):
                    print(f"Página {i + 1}:\n")
                    print(pagina.get_text())
                    print("\n" + "-" * 80 + "\n")
        except Exception as e:
            print(f"⚠️ Erro ao processar '{arquivo}': {e}")
            
            
scrapping_pdf()


--------------------------------------------------------------------------------
Processando: M:\Thiago\WebScrapping\teste_pdf\2UF0EVUTQGB-AFN4ZV5.pdf
Proposta:  09715935
Valor Compra e Venda:  R$ 338.000,00
Matrícula:  143920
Logradouro:  R LUIS LAFRAIA
Número:  162
Andar:  
Complemento:  RESIDENCIA 1 1
Bairro:  ESPLANADA DOS BARREIROS
Cidade:  SAO VICENTE
UF:  SP
Latitude:  -23.960000
Longitude:  -46.400410
Coordenadas:  -23.959906470492072, -46.40004045935697
Área Total:  0,00
Testada (Frente):  0,00
Fração Ideal:  24,158600
N° de Pavimentos:  0
N° Unidades Por Andar:  0
Nº Total de Unidades no condomínio:  N° de Elevadores: 0
N° de Elevadores:  0
Andares/Pavimentos:  
Uso do Edifício:  Residencial
Uso do Imóvel Avaliando:  Residencial
Cobertas:  1
Descobertas:  1
Privativas:  1
Fechamento das Paredes:  Alvenaria
Total de Banheiros:  2
Fachada Principal:  Pintura
Esquadrias:  Alumínio
N° Pavimentos da Unidade:  1
Nº Dormitórios:  3
Área Privativa:  99,20
Área Comum:  0,00
Área Total